In [1]:
import dfBasics
import common
import encoder
import pfAdapt
#import charts


In [124]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org fastparquet

Could not fetch URL https://pypi.org/simple/fastparquet/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/fastparquet/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1125)'))) - skipping
ERROR: Could not find a version that satisfies the requirement fastparquet
ERROR: No matching distribution found for fastparquet
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1125)'))) - skipping


In [126]:
import os
os.environ['http_proxy'] = "http://172.30.12.56:3128" 
os.environ['https_proxy'] = "https://172.30.12.56:3128"  
!pip install pyarrow
#!pip3 install fastparquet
#!conda uninstall fastparquet
#!conda config --add channels conda-forge
#!conda install -y -c conda-forge fastparquet

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1125)'))) - skipping


In [2]:
import pandas as pd
from pyspark.sql import functions

In [3]:
def getCountDF(pf,column,hashes):
    dft = pd.DataFrame(columns=[column, 'count'])
    i=0
    for hash in hashes:
        pfall=pf[pf[column] == hash]
        num=len(pfall)
        if num > 1:
            dft.loc[i] = [hash] + [num]
            i=i+1
    return dft
    #return dft.sort_values('count')

def usedcolumns(tb,row):
    col = []
    for column in tb.columns:
        if tb.iloc[row][column] == None :
            col.append(column)
    return col

def diffcolumns(tb):
    col = []
    for column in tb.columns:
        if tb.iloc[0][column] != tb.iloc[1][column] :
            col.append(column)
    return col

def printtt1():    
    col = []
    for index, row in tt.iterrows():
        tb = pfall[pfall['CGLOBALMESSAGEID'] == row['CGLOBALMESSAGEID']]
        for bindex, brow in tb.iterrows():
            if pfall.loc[bindex]['CSLADELIVERYTIME'] < 0 :
                #print (str(index) + ' ' + str(bindex))
                col.append(bindex)
    return col


In [4]:
def astype(pfall,selected,newtype):
    for each in selected:
        pfall[each] = pfall[each].astype(newtype)     


# B

In [5]:
selected = [ 'CSTATUS', 'CSERVICE',\
        'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
        'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID']


In [6]:
def encodepfall(pfall,number):
    print('1')
    astype(pfall,['CSTARTTIME','CENDTIME','CSLATAT','CMESSAGETAT2','CSLADELIVERYTIME','CINBOUNDSIZE'] ,int) 
    #del(pfall['CSLABILLINGMONTH'])
    pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
    print('1b')
    ac = pd.unique(pfall['CGLOBALMESSAGEID'])
    print('2')
    tt = getCountDF(pfall,'CGLOBALMESSAGEID',ac)
    print('3')
    col = printtt() 
    print('4')
    pfall = pfall.drop(col)
    print('5')
    astype(pfall,selected,str) 
    encoder.encode(pfall,selected)
    
    print('6')
    pfall = pfall.drop_duplicates()
    print('7')
    pfall.to_parquet('/tmp/sla_' + number + '.parquet', engine='fastparquet', compression='GZIP')
    return pfall

# Main

In [7]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
       'CSLABILLINGMONTH', 'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
       'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
       'CMESSAGETAT2', 'CSLADELIVERYTIME']
# withot 'CSLABILLINGMONTH'
def get_columns_2():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
            'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
           'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
           'CMESSAGETAT2', 'CSLADELIVERYTIME']
    return columns
columns = get_columns_2()
#to count messages sent
#columns = [ 'CSTARTTIME', 'CSENDERENDPOINTID']

In [8]:
sparkSession = dfBasics.getSparkSession()

In [ ]:
#len(pfall), len(ac)
#pfall.head()
#pfall.dtypes
#len(mdcountsall)

In [ ]:
#pfall[pfall['CGLOBALMESSAGEID'] == mdcountsall.index.get_level_values(0)[0]]
#pfall['CGLOBALMESSAGEID'] == -9209718302575659389

In [ ]:
#len(pfall[pfall['CSLADELIVERYTIME'] == -1])
#mdcountsall.index.get_level_values(0)[0]

In [9]:
def printtt(pfall,tt):    
    col = []
    #col2 = []
    i = 0
    for index in tt.index.get_level_values(0):
        tb = pfall[pfall['CGLOBALMESSAGEID'] == index]
        findex = []
        for bindex, brow in tb.iterrows():   
            if pfall.loc[bindex]['CSLADELIVERYTIME'] < 0 :
                #print (str(index) + ' ' + str(bindex))
                #col.append(bindex)
                findex.append(bindex)
        if(len(findex) == 1):
            col.append(findex[0])   
        elif(len(findex) == 2):
            if (pfall.iloc[findex[0]]['CENDTIME'] > pfall.iloc[findex[1]]['CENDTIME']):
                col.append(findex[0])
            else:
                col.append(findex[1])
        else:
            if (tb.iloc[0]['CMESSAGETAT2'] > tb.iloc[1]['CMESSAGETAT2']):
                col.append(tb.index[0])
            else:
                col.append(tb.index[1]) 
    return col



In [ ]:
#pfall.iloc[g[1][0]]['CENDTIME'],pfall.iloc[g[1][1]]['CENDTIME']

In [ ]:
#len(f), len(mdcountsall),f
#line= '1580137124017'

#from py4j.java_gateway import Py4JJavaError


In [ ]:
# read timestamps from file
with open("/tmp/slatimestamps.txt", "r") as file:
    lines = file.read().split('\n')

# slatimestamps

In [10]:
filetimestamps = list(sparkSession.read.text('hdfs://172.30.17.145:8020/user/admin/slatimestamps.txt').select('value').toPandas()['value'])

In [ ]:
filetimestamps[0]

In [11]:
def getencodedpfall(line) :
    try:
        df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/' + line +  '/*').select(columns).dropDuplicates()
        #print('df.toPandas')
        pfall = df.toPandas() 
        #print('1')
        astype(pfall,['CSTARTTIME','CENDTIME','CSLATAT','CMESSAGETAT2','CSLADELIVERYTIME','CINBOUNDSIZE'] ,int) 
        #del(pfall['CSLABILLINGMONTH'])
        pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
        #print('1b')
        #ac = pd.unique(pfall['CGLOBALMESSAGEID'])
        mdcountsall = pfall.groupby(['CGLOBALMESSAGEID','CSTATUS'])['CSTARTTIME'].count()
        mdcountsall = mdcountsall[mdcountsall > 1]
        f = printtt(pfall,mdcountsall)
        pfall = pfall.drop(f)
        astype(pfall,selected,str) 
        encoder.encode(pfall,selected)
        pfall.to_parquet('/tmp/sla_' + line + '.parquet', engine='fastparquet', compression='GZIP')
    except Exception as e:
        print("does not exist:" + line)


In [99]:
line = filetimestamps[0]
pfall = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/' + line +  '/*').select(columns).dropDuplicates().toPandas()
astype(pfall,['CSTARTTIME','CENDTIME','CSLATAT','CMESSAGETAT2','CSLADELIVERYTIME','CINBOUNDSIZE'] ,int) 
pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
mdcountsall = pfall.groupby(['CGLOBALMESSAGEID','CSTATUS'])['CSTARTTIME'].count()
mdcountsall = mdcountsall[mdcountsall > 1]
f = printtt(pfall,mdcountsall)
pfall = pfall.drop(f)
astype(pfall,selected,str) 
encoder.encode(pfall,selected)

In [100]:
# check for duplicates
g=pfall.groupby('CGLOBALMESSAGEID')['CGLOBALMESSAGEID'].value_counts()
_duplicates = list(g.where(g>1).dropna().index.get_level_values(0))
 
_drop_index = []
for id in _duplicates:
    index = pfall[pfall['CGLOBALMESSAGEID'] == id].sort_values('CENDTIME').index
    _drop_index.append(index[0])
     
# drop duplicates    
pfall = pfall.drop(_drop_index)    

In [127]:
#pfall.to_parquet('/tmp/sla_' + line + '.parquet', engine='fastparquet', compression='GZIP')
pfall.to_parquet('/tmp/sla_' + line + '.parquet', engine='pyarrow', compression='GZIP')

In [128]:
!ls -l /tmp

total 275888
drwxr-xr-x 66 jovyan users      4096 Mar 31 08:50 blockmgr-cbbf292a-2a1f-4307-9f5a-ba669ad6f595
drwxr-xr-x  1 jovyan users        18 Mar 31 08:49 hsperfdata_jovyan
drwxr-xr-x  2 root   root         18 Mar 29 10:24 hsperfdata_root
-rw-r--r--  1 jovyan users    203408 Mar 31 08:50 liblz4-java-11720395175186768582.so
-rw-r--r--  1 jovyan users         0 Mar 31 08:50 liblz4-java-11720395175186768582.so.lck
drwx------  2 jovyan users         6 Mar 31 13:20 pip-ephem-wheel-cache-rm3pmrj_
drwx------  2 jovyan users         6 Mar 31 13:20 pip-install-dcrnyug8
drwx------  2 jovyan users         6 Mar 31 13:20 pip-req-tracker-6ffcown5
-rw-r--r--  1 jovyan users 281976658 Mar 31 14:25 sla_1580137124017.parquet
drwxr-xr-x  2 jovyan users     36864 Mar 29 15:45 sla.parquet
-rwxr--r--  1 jovyan users    281680 Mar 31 08:50 snappy-1.1.8-922fdeea-d9f2-479a-826e-363bf94a577c-libsnappyjava.so
drwxr-xr-x  2 jovyan users         6 Mar 31 08:49 spark-5403d47d-c602-47f0-95f3-f5538619f536
drwx--

In [96]:
#pfall.loc[_drop_index]
#pfall = pfall.drop(pfall.index[_drop_index])
pfall = pfall.drop(_drop_index)

In [103]:
#pfall[pfall['CGLOBALMESSAGEID'] == _duplicates[0]].sort_values('CENDTIME')
pfall[pfall['CGLOBALMESSAGEID'] == _duplicates[8]]

,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSENDERPROTOCOL,CSENDERENDPOINTID,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME
8313239,-4970793766766447330,1579932039081,1579932045247,11,5,0,2191,628,2,2257,5888,6166,1579932044969


In [97]:
def cols_having_unique(df):
    my_cols = []
    for col in df.columns:
        if df[col].nunique(dropna=False) > 1:
            my_cols.append(col)
    return df[my_cols].copy()

#pfall[pfall['CGLOBALMESSAGEID'] == _duplicates[0]].diff()
cols_having_unique(pfall[pfall['CGLOBALMESSAGEID'] == _duplicates[1]])


""
3983845


In [52]:
#encoder.encode(pfall,selected)
1580115262306 > 1580115260910


True

In [ ]:
encoder.encode(pfall,[selected[0]])

In [ ]:
getencodedpfall(filetimestamps[0])

In [ ]:
#line = lines[0]
#pfall = getencodedpfall(line)

In [ ]:
for line in lines:
    getencodedpfall(line)
    

In [ ]:
#df = sparkSession.read.parquet('/tmp/sla_*.parquet')
#pfall = df.toPandas()